Gerador do Dataset com filtro par aimagens sem label ou corrompidas


In [ ]:
# ==============================================================================
# GERADOR DE DATASET FINAL - COM TRAVA DE SEGURANÇA (FAIL FAST)
# ==============================================================================

import os
import shutil
import cv2
import re
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import random
import yaml
import ast
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- CONFIGURAÇÃO ---
RAW_DATASET_PATH = Path("/content/drive/MyDrive/meus_modelos_ia/datasets/BID Dataset")
OUTPUT_PATH = Path("/content/drive/MyDrive/meus_modelos_ia/datasets/BID_YOLO_Universal_Final")

# Limite de falhas consecutivas antes de abortar
MAX_CONSECUTIVE_FAILURES = 50

CLASSES = [
    'cpf', 'nome', 'data_nasc', 'rg_numero', 'filiacao',
    'categoria', 'validade', 'data_exp', 'naturalidade'
]
CLASS_MAP = {name: idx for idx, name in enumerate(CLASSES)}

# --- FUNÇÕES ---

def normalize_yolo(box, img_w, img_h):
    x, y, w, h = box
    center_x = (x + w / 2) / img_w
    center_y = (y + h / 2) / img_h
    norm_w = w / img_w
    norm_h = h / img_h
    return max(0, min(1, center_x)), max(0, min(1, center_y)), max(0, min(1, norm_w)), max(0, min(1, norm_h))

def parse_line(line):
    try:
        parts = line.strip().split(',', 4)
        if '[' in parts[0]: # Polígono
            list_split = line.strip().split('],')
            if len(list_split) < 2: return None

            xs_str = list_split[0] + ']'
            xs_str = xs_str.replace('[ ', '[').strip()
            ys_str = list_split[1]
            if '[' not in ys_str: ys_str = ys_str[ys_str.find('['):]
            ys_str = ys_str.split(']')[0] + ']'

            xs = ast.literal_eval(xs_str)
            ys = ast.literal_eval(ys_str)

            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            text_part = line.split(',')[-1].strip().upper()
            return {'box': (x_min, y_min, x_max - x_min, y_max - y_min), 'text': text_part}
        else: # Retângulo
            if len(parts) < 5: return None
            x, y, w, h = map(int, [p.strip() for p in parts[:4]])
            text = parts[4].strip().upper()
            return {'box': (x,y,w,h), 'text': text}
    except: return None

def find_nearest_hybrid(anchor_box, candidates, img_w, img_h):
    ax, ay, aw, ah = anchor_box
    acx, acy = ax + aw/2, ay + ah/2
    best_candidate = None
    min_score = float('inf')

    for item in candidates:
        bx, by, bw, bh = item['box']
        bcx, bcy = bx + bw/2, by + bh/2
        dist = np.sqrt((acx - bcx)**2 + (acy - bcy)**2)

        is_below = (by > ay) and (abs(bcx - acx) < (aw + bw))
        is_right = (bx > ax) and (abs(bcy - acy) < (ah + bh))

        score = dist
        if is_below: score *= 0.8
        elif is_right: score *= 0.9
        else: score *= 3.0

        if dist < (max(img_w, img_h) * 0.4):
            if score < min_score:
                min_score = score
                best_candidate = item
    return best_candidate

def parse_gt_file_final(txt_path, img_w, img_h):
    items = []

    # 1. Leitura Robusta
    lines = []
    try:
        with open(txt_path, 'r', encoding='utf-8') as f: lines = f.readlines()
    except UnicodeDecodeError:
        try:
            with open(txt_path, 'r', encoding='latin-1') as f: lines = f.readlines()
        except: return [], "Erro Fatal de Leitura"
    except: return [], "Erro Arquivo"

    for line in lines:
        if "x, y" in line: continue
        parsed = parse_line(line)
        if parsed: items.append(parsed)

    if not items: return [], "Arquivo Vazio"

    found_labels = []
    used_indices = set()

    # --- NOVIDADE 1: Limpeza para Matching ---
    def clean_key(t): return t.replace('.','').replace('-','').replace(' ','')

    # Mapeamento de Âncoras
    anchors = {}
    keywords = {
        'NOME': ['NOME', 'NOMEDAPESSOA'],
        'CPF': ['CPF', 'INSCRICAO', 'CIC'],
        'RG': ['DOCIDENTIDADE', 'REGISTROGERAL', 'RG', 'GERAL', 'IDENTIDADE'],
        'NASC': ['DATANASCIMENTO', 'NASCIMENTO', 'NASC'],
        'FILIACAO': ['FILIACAO', 'DOCORIGEM', 'FILI'],
        'CAT': ['CATHAB', 'HABILITACAO'],
        'VALIDADE': ['VALIDADE', 'VAL'],
        'DATA_EXP': ['DATAEMISSAO', 'DATAEXPEDICAO', 'EXPEDICAO'],
        'NATURAL': ['NATURALIDADE']
    }

    candidates = []

    for idx, item in enumerate(items):
        raw_txt = item['text']
        clean_txt = clean_key(raw_txt) # Remove pontos e espaços para comparar
        item['idx'] = idx

        is_anchor = False
        for key, words in keywords.items():
            # Verifica se alguma palavra chave está contida no texto limpo
            if any(w in clean_txt for w in words):
                anchors[key] = item
                is_anchor = True
                break

       
        candidates.append(item)

    # Extração Helper
    def add_label(item, class_name):
        if item['idx'] in used_indices: return
        cx, cy, nw, nh = normalize_yolo(item['box'], img_w, img_h)
        if nw < 0.005 or nh < 0.005: return 
        found_labels.append(f"{CLASS_MAP[class_name]} {cx} {cy} {nw} {nh}")
        used_indices.add(item['idx'])


    # CPF
    re_cpf = re.compile(r'\d{3}[\., ]?\d{3}[\., ]?\d{3}[-,\. ]?\d{2}')
    for item in candidates:
       
        txt_nums = re.sub(r'[a-zA-Z]', '', item['text'])
        if len(txt_nums) > 10 and re_cpf.search(txt_nums):
            add_label(item, 'cpf')

    # Datas
    re_data = re.compile(r'\d{2}/\d{2}/\d{4}')
    for item in candidates:
        if re_data.search(item['text']):
            matched = False
           
            for anchor_name, class_name in [('NASC', 'data_nasc'), ('VALIDADE', 'validade'), ('DATA_EXP', 'data_exp')]:
                if anchor_name in anchors:
                    cand = find_nearest_hybrid(anchors[anchor_name]['box'], [item], img_w, img_h)
                    if cand:
                        add_label(item, class_name); matched = True; break

            
            if not matched:
                try:
                    parts = item['text'].split('/')
                    ano = int(parts[-1][:4])
                    if ano < 2015: 
                         add_label(item, 'data_nasc')
                except: pass

    
    pairs = [('NOME', 'nome'), ('RG', 'rg_numero'), ('CAT', 'categoria'), ('NATURAL', 'naturalidade')]
    for anchor_key, class_name in pairs:
        if anchor_key in anchors:
            best = find_nearest_hybrid(anchors[anchor_key]['box'], candidates, img_w, img_h)
            if best:
                if class_name == 'nome' and len(best['text']) < 3: continue
                add_label(best, class_name)

   
    if 'FILIACAO' in anchors:
        potential = []
        abox = anchors['FILIACAO']['box']
        for item in candidates:
            if item['idx'] in used_indices: continue
            if find_nearest_hybrid(abox, [item], img_w, img_h): potential.append(item)
        potential.sort(key=lambda x: x['box'][1])
        for p in potential[:2]:
            if len(p['text']) > 3: add_label(p, 'filiacao')

    if not found_labels:
        return [], "Sem labels (Regex e Âncoras falharam)"

    return found_labels, "OK"

   
    def add_label(item, class_name):
        if item['idx'] in used_indices: return
        cx, cy, nw, nh = normalize_yolo(item['box'], img_w, img_h)
        if nw < 0.01 or nh < 0.01: return
        found_labels.append(f"{CLASS_MAP[class_name]} {cx} {cy} {nw} {nh}")
        used_indices.add(item['idx'])

    re_cpf = re.compile(r'\d{3}[\.,]?\d{3}[\.,]?\d{3}[-,\.]?\d{2}')
    re_data = re.compile(r'\d{2}/\d{2}/\d{4}')

    
    for item in candidates:
        if re_cpf.search(item['text']): add_label(item, 'cpf')

    for item in candidates:
        if re_data.search(item['text']):
            matched = False
            for anchor_name, class_name in [('NASC', 'data_nasc'), ('VALIDADE', 'validade'), ('DATA_EXP', 'data_exp')]:
                if anchor_name in anchors:
                    cand = find_nearest_hybrid(anchors[anchor_name]['box'], [item], img_w, img_h)
                    if cand:
                        add_label(item, class_name); matched = True; break
            if not matched and 'NASC' in anchors: add_label(item, 'data_nasc')

    pairs = [('NOME', 'nome'), ('RG', 'rg_numero'), ('CAT', 'categoria'), ('NATURAL', 'naturalidade')]
    for anchor_key, class_name in pairs:
        if anchor_key in anchors:
            best = find_nearest_hybrid(anchors[anchor_key]['box'], candidates, img_w, img_h)
            if best:
                if class_name == 'nome' and len(best['text']) < 3: continue
                add_label(best, class_name)

    if 'FILIACAO' in anchors:
        potential = []
        abox = anchors['FILIACAO']['box']
        for item in candidates:
            if item['idx'] in used_indices: continue
            if find_nearest_hybrid(abox, [item], img_w, img_h): potential.append(item)
        potential.sort(key=lambda x: x['box'][1])
        for p in potential[:2]:
            if len(p['text']) > 3: add_label(p, 'filiacao')

    if not found_labels:
        return [], "Labels extraídos = 0 (Lógica falhou)"

    return found_labels, "OK"

# ==============================================================================
# EXECUÇÃO COM MONITORAMENTO
# ==============================================================================
def create_final_dataset_safe():
    if OUTPUT_PATH.exists(): shutil.rmtree(OUTPUT_PATH)

    for split in ['train', 'val']:
        (OUTPUT_PATH / 'images' / split).mkdir(parents=True, exist_ok=True)
        (OUTPUT_PATH / 'labels' / split).mkdir(parents=True, exist_ok=True)

    print(f"🚀 Iniciando Dataset Seguro...")
    print(f"🛑 Se ocorrerem {MAX_CONSECUTIVE_FAILURES} erros seguidos, o script para.")

    all_images = list(RAW_DATASET_PATH.rglob("*_in.jpg"))
    random.shuffle(all_images)

    split_idx = int(len(all_images) * 0.8)
    count_success = 0
    consecutive_fails = 0

    pbar = tqdm(all_images)
    for idx, img_path in enumerate(pbar):
        # 1. Checagem de Segurança (Fail Fast)
        if consecutive_fails >= MAX_CONSECUTIVE_FAILURES:
            print("\n" + "!"*60)
            print(f"🚨 ABORTANDO! {MAX_CONSECUTIVE_FAILURES} falhas consecutivas detectadas.")
            print("   Algo está errado com os dados ou com o script.")
            print("   Verifique as mensagens de erro acima.")
            print("!"*60)
            break

        subset = 'train' if idx < split_idx else 'val'
        gt_path = img_path.parent / img_path.name.replace('_in.jpg', '_gt_ocr.txt')

        if not gt_path.exists():
            consecutive_fails += 1
            continue

        try:
            img = cv2.imread(str(img_path))
            if img is None:
                consecutive_fails += 1; continue
            h, w = img.shape[:2]
        except:
            consecutive_fails += 1; continue

        # 2. Processamento com Retorno de Status
        labels, status = parse_gt_file_final(gt_path, w, h)

        if labels:
            # SUCESSO
            try:
                shutil.copy(img_path, OUTPUT_PATH / 'images' / subset / img_path.name)
                with open(OUTPUT_PATH / 'labels' / subset / img_path.name.replace('.jpg', '.txt'), 'w') as f:
                    f.write('\n'.join(labels))
                count_success += 1
                consecutive_fails = 0 # Reseta o contador se der certo
                pbar.set_description(f"✅ Sucessos: {count_success}")
            except Exception as e:
                print(f"❌ Erro ao salvar: {e}")
                consecutive_fails += 1
        else:
            # FALHA
            consecutive_fails += 1
            # Imprime o erro apenas para as primeiras falhas para não poluir
            if consecutive_fails < 5 or consecutive_fails % 10 == 0:
                print(f"⚠️ Falha em {img_path.name}: {status}")
            pbar.set_description(f"⚠️ Falhas Seguidas: {consecutive_fails}")

    # Relatório Final
    if consecutive_fails < MAX_CONSECUTIVE_FAILURES:
        # Cria YAML apenas se não abortou
        yaml_content = {
            'path': str(OUTPUT_PATH), 'train': 'images/train', 'val': 'images/val',
            'names': {i: n for i, n in enumerate(CLASSES)}
        }
        with open(OUTPUT_PATH / 'data.yaml', 'w') as f: yaml.dump(yaml_content, f)

        print(f"\n✅ Finalizado!")
        print(f"📊 Imagens Processadas com Sucesso: {count_success}")
        print(f"📂 Salvo em: {OUTPUT_PATH}")
    else:
        print("\n❌ Processo abortado. Dataset incompleto.")

create_final_dataset_safe()

🚀 Iniciando Dataset Seguro...
🛑 Se ocorrerem 50 erros seguidos, o script para.


  0%|          | 0/28800 [00:00<?, ?it/s]

⚠️ Falha em 00022974_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00025704_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00023996_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00007979_in.jpg: Arquivo Vazio
⚠️ Falha em 00008093_in.jpg: Arquivo Vazio
⚠️ Falha em 00009500_in.jpg: Arquivo Vazio
⚠️ Falha em 00009741_in.jpg: Arquivo Vazio
⚠️ Falha em 00022703_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00025441_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00010157_in.jpg: Arquivo Vazio
⚠️ Falha em 00024991_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00009974_in.jpg: Arquivo Vazio
⚠️ Falha em 00023875_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00007913_in.jpg: Arquivo Vazio
⚠️ Falha em 00025408_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00025131_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00024483_in.jpg: Sem labels (Regex e Âncoras falharam)
⚠️ Falha em 00008291_in.jpg: 

Gerador do data.yml para o treinamento do modelo YOLO

In [ ]:

from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

import yaml
import os
from pathlib import Path

# Configurações
DATASET_PATH = Path("/content/drive/MyDrive/meus_modelos_ia/datasets/BID_YOLO_Universal_Final")
CLASSES = [
    'cpf', 'nome', 'data_nasc', 'rg_numero', 'filiacao',
    'categoria', 'validade', 'data_exp', 'naturalidade'
]

# Conteúdo do arquivo YAML
yaml_content = {
    'path': str(DATASET_PATH),  # Caminho raiz
    'train': 'images/train',    # Subpasta de treino
    'val': 'images/val',        # Subpasta de validação
    'names': {i: name for i, name in enumerate(CLASSES)}
}

# Criar o arquivo na marra
if DATASET_PATH.exists():
    yaml_file = DATASET_PATH / 'data.yaml'

    with open(yaml_file, 'w') as f:
        yaml.dump(yaml_content, f)

    print(f"✅ Arquivo salvo com sucesso em: {yaml_file}")
    print("Conteúdo:")
    print(yaml_content)
else:
    print(f"❌ ERRO GRAVE: A pasta do dataset não existe: {DATASET_PATH}")
    print("Verifique se o Google Drive está montado e se o nome da pasta está correto.")

Mounted at /content/drive
✅ Arquivo salvo com sucesso em: /content/drive/MyDrive/meus_modelos_ia/datasets/BID_YOLO_Universal_Final/data.yaml
Conteúdo:
{'path': '/content/drive/MyDrive/meus_modelos_ia/datasets/BID_YOLO_Universal_Final', 'train': 'images/train', 'val': 'images/val', 'names': {0: 'cpf', 1: 'nome', 2: 'data_nasc', 3: 'rg_numero', 4: 'filiacao', 5: 'categoria', 6: 'validade', 7: 'data_exp', 8: 'naturalidade'}}


Codigo para comprimir dataset Para acelerar o treinamento e impedir que o colab se desligue pelo limite de uso da conta gratuita


In [ ]:
import zipfile
import os
from tqdm.notebook import tqdm
from google.colab import drive

drive.mount('/content/drive')


SOURCE_FOLDER = '/content/drive/MyDrive/meus_modelos_ia/datasets/BID_YOLO_Universal_Final'


OUTPUT_ZIP_FILE = '/content/drive/MyDrive/meus_modelos_ia/datasets/dataset_completo.zip'

def zip_dataset_com_progresso(source_path, output_path):
    print(f"📦 Iniciando compactação de: {source_path}")

 
    file_count = 0
    for root, dirs, files in os.walk(source_path):
       
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        for file in files:
            if not file.startswith('.'):
                file_count += 1

    print(f"📄 Total de arquivos encontrados: {file_count}")

    # 2. Criar o ZIP
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Barra de progresso
        with tqdm(total=file_count, unit='file') as pbar:
            for root, dirs, files in os.walk(source_path):
                # Remove pastas ocultas (.ipynb_checkpoints, etc)
                dirs[:] = [d for d in dirs if not d.startswith('.')]

                for file in files:
                    if file.startswith('.'): # Pula arquivos ocultos
                        continue

                    file_path = os.path.join(root, file)

                    # Define o nome do arquivo DENTRO do zip (caminho relativo)
                    # Ex: se o arquivo é /content/drive/.../images/train/1.jpg
                    # No zip ele vira images/train/1.jpg
                    arcname = os.path.relpath(file_path, start=source_path)

                    zipf.write(file_path, arcname)
                    pbar.update(1)

    # 3. Verificar tamanho final
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ Sucesso! Arquivo salvo em: {output_path}")
    print(f"📊 Tamanho Final: {size_mb:.2f} MB")

# Executar
zip_dataset_com_progresso(SOURCE_FOLDER, OUTPUT_ZIP_FILE)